# Setup

In [113]:
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM
import numpy as np
import torch
from pprint import pprint
import logging
import re

transformers.logging.set_verbosity_error()

def print_header(header):
    print("\n\n----------------------------------------------------------")
    print(header)
    print("----------------------------------------------------------")

models = {
    # 'BERT': 'bert-base-uncased',
    'BERT (whole word)': 'bert-large-uncased-whole-word-masking',
    'GPT': 'gpt2',
}

horse_sentence_dict = {
    'NP/S': {
        'ambiguous': "The horse raced past the barn [MASK]",
        'un-ambiguated': "The horse raced past the barn, [MASK]"
    }
}

horse_fillers = [
        'fell',
        'is',
        'was',
        'and',
        '.',
    ]

butter_sentence_dict = {
    'NP/S': {
        'ambiguous': "The butter melted in the pan [MASK]",
        'un-ambiguated': "The butter melted in the pan, [MASK]",
    }
}

butter_fillers = [
        'smelled',
        'smells',
        'is',
        'was',
        'and',
        '.',
    ]

politician_sentence_dict = {
    'NP/S': {
        'ambiguous': 'The corrupt politician mentioned the bill [MASK]',
        'un-ambiguated': 'The corrupt politician that mentioned the bill [MASK]',
    },
    'NP/Z': {
        'ambiguous': 'After the corrupt politician signed the bill [MASK]',
        'un-ambiguated': 'After the corrupt politician signed, the bill [MASK]',
    },
    'MVRR': {
        'ambiguous': 'The corrupt politician handed the bill [MASK]',
        'un-ambiguated': 'The corrupt politician who was handed the bill [MASK]',
    },
}

politician_fillers = {
    'incorrect': [
            'and',
            '.',
            'to',
    ],
    'correct': [
        'is',
        'was',
        'received'
    ],
}

bert, mask = setup('BERT (whole word)')

# Behavior Exploration -- BERT

In [114]:
def setup(model):
    tokenizer = AutoTokenizer.from_pretrained(models[model])
    model = AutoModelForMaskedLM.from_pretrained(models[model])
    bert = pipeline("fill-mask", model=model, tokenizer=tokenizer)
    mask = bert.tokenizer.mask_token
    return bert, mask

def runBERT(bert, sentence_dict, fillers):
    filler_results = {}
    top_preds = {}
    for sentence_type in sentence_dict:
        top_preds[sentence_type] = {}
        filler_results[sentence_type] = {}
        for clarity in ['ambiguous', 'un-ambiguated']:
            sentence = sentence_dict[sentence_type][clarity]
            outputs = bert(sentence, top_k=12)
            top_preds[sentence_type][clarity] = [(output["token_str"], output['score']) for output in outputs]
            filler_results[sentence_type][clarity] = {}
            for accuracy in fillers:
                filler_results[sentence_type][clarity][accuracy] = {}
                for filler in fillers[accuracy]:
                    filler_results[sentence_type][clarity][accuracy][filler] = bert(sentence, targets=[filler])[0]["score"]
    return filler_results, top_preds
    

In [115]:
filler_results, top_preds = runBERT(bert, politician_sentence_dict, politician_fillers)

print_header('Filler Results')
pprint(filler_results, sort_dicts=False)

# print_header('Model Predictions')
# pprint(top_preds, sort_dicts=False)




----------------------------------------------------------
Filler Results
----------------------------------------------------------
{'NP/S': {'ambiguous': {'incorrect': {'and': 0.0013153573963791132,
                                      '.': 0.7417082190513611,
                                      'to': 0.0021446088794618845},
                        'correct': {'is': 0.0003153316501993686,
                                    'was': 0.0009200123022310436,
                                    'received': 2.3071584109857213e-06}},
          'un-ambiguated': {'incorrect': {'and': 0.00040642256499268115,
                                          '.': 0.22318512201309204,
                                          'to': 0.0011608753120526671},
                            'correct': {'is': 0.05572093278169632,
                                        'was': 0.15602436661720276,
                                        'received': 2.1722347810282372e-05}}},
 'NP/Z': {'ambiguous': {'incorrect

In [116]:
def get_ratios(filler_results, fillers):
    ratios = {}
    for sentence_type in filler_results:
        ratios[sentence_type] = {}
        for clarity in ['ambiguous', 'un-ambiguated']:
            correct_mean = 0
            incorrect_mean = 0
            for filler in fillers['correct']:
                correct_mean += filler_results[sentence_type][clarity]['correct'][filler]
            correct_mean /= len(filler_results[sentence_type][clarity]['correct'])
            for filler in fillers['incorrect']:
                incorrect_mean += filler_results[sentence_type][clarity]['incorrect'][filler]
            incorrect_mean /= len(filler_results[sentence_type][clarity]['incorrect'])
            ratios[sentence_type][clarity] = correct_mean/incorrect_mean
    return ratios


In [117]:
ratios = get_ratios(filler_results, politician_fillers)
pprint(ratios)

{'MVRR': {'ambiguous': 2.1342639492010513e-05,
          'un-ambiguated': 0.01974517359528995},
 'NP/S': {'ambiguous': 0.001660901706770999,
          'un-ambiguated': 0.9422235446907513},
 'NP/Z': {'ambiguous': 0.0008670076072932954,
          'un-ambiguated': 1.6903761795611767}}


# Behavior Exploration -- GPT2

In [118]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np 

def score(model, tokens_tensor):
    loss=model(tokens_tensor, labels=tokens_tensor)[0]
    return np.exp(loss.cpu().detach().numpy())

def runGPT(sentence_dict):
    model = GPT2LMHeadModel.from_pretrained(models['GPT'])
    tokenizer = GPT2Tokenizer.from_pretrained(models['GPT'])
    filler_results = {}
    top_preds = {}
    model.eval()
    for sentence_type in sentence_dict:
        for sentence in sentence_type['sentences']:
            # text = sentence.replace('[MASK]', '')
            # with torch.no_grad():
                # outputs = model(tokenizer.encode(text, add_special_tokens=False, return_tensors="pt"))
                # predictions = outputs[0][0, -1, :]
                # print(len(predictions))
                # print([tokenizer.decode([pred.item()]) for pred in predictions])

            # next_token_logits = outputs[0]
            # print(next_token_logits)
            
            filler_results[sentence] = {}
            for filler in sentence_type['fillers']:
                tokens_tensor = tokenizer.encode(sentence.replace('[MASK]', filler), add_special_tokens=False, return_tensors="pt")
                filler_results[sentence][filler] = score(model, tokens_tensor)
    return filler_results, top_preds

In [119]:
filler_results, top_preds = runGPT(sentence_dict)
print_header("Filler Results")
pprint(filler_results, sort_dicts=False)
print_header("Model Predictions")
pprint(top_preds, sort_dicts=False)



----------------------------------------------------------
Filler Results
----------------------------------------------------------
{'The corrupt politician mentioned the bill [MASK]': {'is': 723.888,
                                                      'was': 617.42737,
                                                      'and': 574.95264,
                                                      '.': 1431.8153,
                                                      'to': 592.2486,
                                                      'received': 1414.098},
 'The corrupt politician that mentioned the bill [MASK]': {'is': 538.2617,
                                                           'was': 508.98694,
                                                           'and': 585.8596,
                                                           '.': 1188.5912,
                                                           'to': 596.9901,
                                                         